# Simulation of microservices

This simulation attempts to balance the size of a micro service

## Microservice

The size of a [microservice](https://en.wikipedia.org/wiki/Microservices) is related to the number of features it provides.

In [140]:
from typing import List, Tuple, Set
from fractions import Fraction

In [166]:
class MicroService:
    def __init__(self):
        self.feature_count = 0
    
    def set_feature_count(self, count: int):
        self.feature_count = count
        return self
        
    def __str__(self):
        return "MicroService: features: {}".format(self.feature_count)
           

In [167]:
micro1 = MicroService()
micro1.set_feature_count(3)
print(micro1)

MicroService: features: 3


## ServiceSuite

A service suite represents an aggregation of services used by a department.



In [168]:
from random import randint
class ServiceSuite:
    def __init__(self):
        self.services = []
           
    def __str__(self):
        return "ServiceSuite: count: {}, features: {}".format(self.get_service_count(), self.get_feature_count())
        
    def add_microservice(self, microservice: MicroService):
        self.services.append(microservice)
        return self
        
    def get_feature_count(self)->int:
        return sum([s.feature_count for s in self.services])
    
    def get_service_count(self)->int:
        return len(self.services)
    
    def random_suite(self, target_feature_count: int, min_features_by_service: int):
        """
        This creates a random suite that will produce the desired target number of features accross
        different services
        """
        remaining_feature_count = target_feature_count
        while remaining_feature_count >= min_features_by_service:
            service_feature_count = randint(min_features_by_service, remaining_feature_count)
            if remaining_feature_count - service_feature_count <= min_features_by_service:
                service_feature_count = remaining_feature_count
            self.add_microservice(MicroService().set_feature_count(service_feature_count))
            remaining_feature_count -= service_feature_count
        return self
    
    def get_summary(self)->str:
        features_by_service = " ".join(sorted([str(s.feature_count) for s in self.services]))
        return "{} services, features: {}".format(self.get_service_count(), features_by_service)
    
    def __eq__(self, other):
        return self.get_summary() == other.get_summary()

    def __hash__(self):
        return hash(self.get_summary())
 
 
   

In [169]:
serviceSuite1 = ServiceSuite()
serviceSuite1.random_suite(target_feature_count = 10, min_features_by_service = 3)
print(serviceSuite1)
print(serviceSuite1.get_summary())

ServiceSuite: count: 2, features: 10
2 services, features: 3 7


## Specifications

### Developement time

The developer time in hours

In [170]:
class DevTimeSpecs:
    def __init__(self):
        self.hours_per_service = 1
        self.hours_per_feature = 1
        self.penalty_exp = 1.1
    
    def set_hours_per_service(self, hours: float):
        self.hours_per_service = hours
        return self

    def set_hours_per_feature(self, hours: float):
        self.hours_per_feature = hours
        return self
    
    def set_penalty_exp(self, penalty_exp: float):
        self.penalty_exp = penalty_exp
        return self
   
    def __str__(self):
        return "DevTimeSpecs: h/service : {}, h/feature: {}".format(self.hours_per_service, self.hours_per_feature)
        
    def get_devtime(self, service: MicroService)->float:
        return self.hours_per_service + service.feature_count*(self.penalty_exp**service.feature_count)*self.hours_per_feature
   
    def get_devtime_for_suite(self, suite: ServiceSuite)->float:
        return sum([self.get_devtime(service) for service in suite.services])


### Business specifications

We can simplify the business specifications as a number of features to provide.

In [171]:
class BusinessSpecs:
    def __init__(self):
        self.feature_count = 0
        self.min_features_by_service = 3
        self.dev_time_specs = None
    
    def set_feature_count(self, count: int):
        self.feature_count = count
        return self
    
    def set_dev_time_specs(self, dev_time_specs: DevTimeSpecs):
        self.dev_time_specs = dev_time_specs
        return self
    
    def set_min_features_by_service(self, min_features_by_service: int):
        self.min_features_by_service = min_features_by_service
        return self
       
    def __str__(self):
        return "BusinessSpecs: features: {}, min feat./serv. {}, dev time: {}".format(self.feature_count, self.min_features_by_service, self.dev_time_specs)
    

In [172]:
businessSpecs1 = BusinessSpecs()
businessSpecs1.set_feature_count(50)
businessSpecs1.set_dev_time_specs(DevTimeSpecs().set_hours_per_service(8).set_hours_per_feature(16).set_penalty_exp(1.05))
print("hours", businessSpecs.dev_time_specs.get_devtime(MicroService().set_feature_count(4)))

hours 85.79240000000001


## Simulation

The library Pandas(https://pandas.pydata.org/) makes it simpler to manipulate data.

In [209]:
import pandas as pd

class Simulation:
    def __init__(self):
        self.business_specs = None
        self.service_suite_set = set([])
        
    def set_business_specs(self, business_specs: BusinessSpecs):
        self.business_specs = business_specs
        return self
    
    def add_service_suite(self, service_suite: ServiceSuite):
        self.service_suite_set.add(service_suite)
        return self
    
    def random_suites(self, count: int):
        target = self.business_specs.feature_count
        min_feat = self.business_specs.min_features_by_service
        for _ in range(count):
            suite = ServiceSuite().random_suite(target_feature_count = target, min_features_by_service = min_feat)
            self.add_service_suite(suite)
        return self
                
    def _to_suite_tuple(self, suite: ServiceSuite):
        return (suite.get_summary(), suite.get_service_count(), self.business_specs.dev_time_specs.get_devtime_for_suite(suite))
        
    def to_list(self):
        return [self._to_suite_tuple(suite) for suite in list(self.service_suite_set)]
    
    def to_dataframe(self):
        return pd.DataFrame(self.to_list(), columns=['Description', 'Services', 'Hours'])
      

In [210]:
simulation1 = Simulation().set_business_specs(businessSpecs1).random_suites(7)
print(simulation1.to_dataframe())

                        Description  Services        Hours
0  4 services, features: 11 12 20 7         4  1684.474655
1   4 services, features: 12 25 6 7         4  2017.590820
2        2 services, features: 42 8         2  5420.901134
3     3 services, features: 13 28 9         3  2395.824139
4          1 services, features: 50         1  9181.919829
5    4 services, features: 35 4 5 6         4  3429.512712
6      3 services, features: 36 6 8         3  3677.849573


In [211]:
simulation_data = simulation1.to_dataframe()

### Visualisation

[Altair](https://altair-viz.github.io/getting_started/overview.html) is a declarative statistical visualization library for Python, based on Vega and Vega-Lite that will need to be [installed](https://altair-viz.github.io/getting_started/installation.html).

In [212]:
import altair as alt

alt.Chart(simulation_data).mark_circle(size=60).encode(
    x='Services',
    y='Hours',
    tooltip=['Description']
).interactive()


alt.Chart(...)